# Growbal Intelligence System - Complete Agent Testing

This notebook tests all agents in the Growbal Intelligence system:
1. **SearchAgent** - Finds candidate profiles based on queries
2. **AdjudicatorAgent** - Evaluates relevance of candidate profiles
3. **SummarizerAgent** - Creates comprehensive summaries of relevant profiles

Each agent is tested individually, and outputs from one agent are used as inputs to the next.

In [ ]:
# Setup imports and environment
import os
import sys
import asyncio
from pprint import pprint
import json
from datetime import datetime
import uuid

# Get the current working directory and navigate to the proper paths
# In Jupyter notebooks, we use os.getcwd() instead of __file__
notebook_dir = os.getcwd()
print(f"Notebook directory: {notebook_dir}")

# Navigate to the project root (growbal) directory
# This is needed because the imports use growbal_intelligence as the base package
if 'testing' in notebook_dir:
    # We're in growbal_intelligence/testing/
    project_root = os.path.dirname(os.path.dirname(notebook_dir))  # Go up two levels
else:
    # Try to find project root by looking for growbal_intelligence directory
    current = notebook_dir
    while current != os.path.dirname(current):  # While not at filesystem root
        if os.path.exists(os.path.join(current, 'growbal_intelligence')):
            project_root = current
            break
        current = os.path.dirname(current)
    else:
        # Fallback: assume we're somewhere in the project
        project_root = os.path.dirname(notebook_dir)

# Add project root to Python path
if project_root not in sys.path:
    sys.path.insert(0, project_root)

print(f"Project root: {project_root}")
print(f"Python path: {sys.path[:3]}...")  # Show first 3 entries

# Import all agents and models using full package paths
from growbal_intelligence.agents.search_agent import SearchAgent
from growbal_intelligence.agents.adjudicator_agent import AdjudicatorAgent
from growbal_intelligence.agents.summarizer_agent import SummarizerAgent
from growbal_intelligence.core.models import (
    SearchAgentInput, SearchAgentOutput,
    AdjudicatorAgentInput, AdjudicatorAgentOutput,
    SummarizerAgentInput, SummarizerAgentOutput,
    ProfileMatch, AdjudicationResult,
    WorkflowState
)
from growbal_intelligence.core.workflow import GrowbalIntelligenceWorkflow

print("All imports successful!")
print(f"Working directory: {notebook_dir}")
print(f"Project root: {project_root}")

Notebook directory: /home/mohammed/Desktop/tech_projects/growbal/growbal_intelligence/testing
Project root: /home/mohammed/Desktop/tech_projects/growbal
Python path: ['/home/mohammed/Desktop/tech_projects/growbal', '/home/mohammed/anaconda3/envs/growbal/lib/python311.zip', '/home/mohammed/anaconda3/envs/growbal/lib/python3.11']...
✅ All imports successful!
📁 Working directory: /home/mohammed/Desktop/tech_projects/growbal/growbal_intelligence/testing
📁 Project root: /home/mohammed/Desktop/tech_projects/growbal


In [ ]:
# Initialize all agents
api_key = os.getenv('ANTHROPIC_API_KEY')
if not api_key:
    print("WARNING: ANTHROPIC_API_KEY not found in environment!")
    print("Please set it or pass it directly to the agents")
    # api_key = "your-api-key-here"

# Initialize agents
search_agent = SearchAgent(api_key=api_key)
adjudicator_agent = AdjudicatorAgent(api_key=api_key)
summarizer_agent = SummarizerAgent(api_key=api_key)

print("All agents initialized successfully!")

# Test query for the entire flow
TEST_QUERY = "I need a immigration and relocation firm that specializes in migration, business and investment migration."
# TEST_QUERY = "I need a list of service providers located in Dubai, Business Bay"
print(f"\nTest Query: {TEST_QUERY}")

✅ All agents initialized successfully!

🔍 Test Query: I need a immigration and relocation firm that specializes in migration, business and investment migration.


## Step 1: Search Agent Testing

The SearchAgent analyzes the query and retrieves candidate profiles using semantic search, tag-based search, or hybrid strategies.

In [ ]:
# Create search input
search_input = SearchAgentInput(
    query=TEST_QUERY,
    max_results=7,  # Reduced from 10 to 7 candidates for better performance
    minimum_similarity=0.5  # No minimum threshold for testing
)

print("SEARCH AGENT TEST")
print("=" * 60)
print(f"Query: {search_input.query}")
print(f"Max Results: {search_input.max_results}")
print(f"Minimum Similarity: {search_input.minimum_similarity}")
print("\nExecuting search...")

# Execute search
search_response = await search_agent.search(search_input)

# Display results
print(f"\nSearch Completed: {search_response.success}")
print(f"Message: {search_response.message}")
print(f"Processing Time: {search_response.processing_time:.2f} seconds")
print(f"Confidence Score: {search_response.confidence_score}")

print("\nSearch Metadata:")
for key, value in search_response.metadata.items():
    print(f"  - {key}: {value}")

# Store search output for next step
search_output = search_response.data

🔍 SEARCH AGENT TEST
Query: I need a immigration and relocation firm that specializes in migration, business and investment migration.
Max Results: 7
Minimum Similarity: 0.5

Executing search...

✅ Search Completed: True
📝 Message: Successfully found 7 candidate profiles using hybrid search
⏱️  Processing Time: 4.84 seconds
🎯 Confidence Score: 1.0

📊 Search Metadata:
  - search_strategy: hybrid
  - extracted_tags: ['immigration', 'relocation', 'migration', 'business migration', 'investment migration']
  - strategy_reasoning: This query contains both specific service tags (immigration, migration types) and requires semantic understanding of the comprehensive service needs. A hybrid search strategy will leverage both tag-based matching and semantic similarity to find the most relevant service providers.
  - similarity_search_query: a professional immigration and relocation service provider specializing in comprehensive migration solutions, including business and investment migration pathw

In [ ]:
# Display search results details
if search_output and search_output.candidate_profiles:
    print(f"\nSEARCH RESULTS SUMMARY")
    print("=" * 60)
    print(f"Total profiles searched: {search_output.total_profiles_searched}")
    print(f"Candidates found: {len(search_output.candidate_profiles)}")
    print(f"Search strategy: {search_output.search_strategy}")
    print(f"Search time: {search_output.search_time_seconds:.2f} seconds")
    
    print("\nCANDIDATE PROFILES:")
    for i, profile in enumerate(search_output.candidate_profiles, 1):  # Show first 5
        print(f"\n--- Profile {i} ---")
        print(f"ID: {profile.profile_id}")
        print(f"Similarity Score: {profile.similarity_score:.3f}")
        print(f"Profile Preview (first 300 chars):")
        print(f"{profile.profile_text[:300]}...")
else:
    print("No search results found!")


📋 SEARCH RESULTS SUMMARY
Total profiles searched: 34
Candidates found: 7
Search strategy: Hybrid search (vector similarity + tag scores combined) with tags: immigration, relocation, migration, business migration, investment migration
Search time: 0.84 seconds

🏢 CANDIDATE PROFILES:

--- Profile 1 ---
ID: 202
Similarity Score: 1.000
Profile Preview (first 300 chars):
=== SERVICE PROVIDER PROFILE ===
Company Name: AIMS Immigration & Relocation Specialist
Provider Type: Company
Country: Singapore
Session Status: inactive
Email Addresses: assessment@aims.sg, daniel@aims.sg
Telephone Numbers: +65 6713 0155
Website: https://aims.sg
LinkedIn: https://www.linkedin.com/...

--- Profile 2 ---
ID: 201
Similarity Score: 1.000
Profile Preview (first 300 chars):
=== SERVICE PROVIDER PROFILE ===
Company Name: Abode Options
Provider Type: Company
Country: UAE
Session Status: inactive
Email Addresses: info@abodeoptions.com, jehan@abodeoptions.com
Telephone Numbers: +971 4 406 9656, +971 4 06 9650, +24

## Step 2: Adjudicator Agent Testing

The AdjudicatorAgent evaluates each candidate profile's relevance to the original query.

In [ ]:
# Create adjudicator input using search results
if search_output and search_output.candidate_profiles:
    adjudicator_input = AdjudicatorAgentInput(
        original_query=TEST_QUERY,
        candidate_profiles=search_output.candidate_profiles,
        relevance_threshold=0.7  # 70% relevance threshold
    )
    
    print("ADJUDICATOR AGENT TEST")
    print("=" * 60)
    print(f"Original Query: {adjudicator_input.original_query}")
    print(f"Candidates to evaluate: {len(adjudicator_input.candidate_profiles)}")
    print(f"Relevance threshold: {adjudicator_input.relevance_threshold}")
    print("\nEvaluating candidate profiles...")
    
    # Execute adjudication
    adjudicator_response = await adjudicator_agent.adjudicate(adjudicator_input)
    
    # Display results
    print(f"\nAdjudication Completed: {adjudicator_response.success}")
    print(f"Message: {adjudicator_response.message}")
    print(f"Processing Time: {adjudicator_response.processing_time:.2f} seconds")
    print(f"Overall Confidence: {adjudicator_response.confidence_score:.2f}")
    
    print("\nAdjudication Metadata:")
    for key, value in adjudicator_response.metadata.items():
        print(f"  - {key}: {value}")
    
    # Store adjudicator output for next step
    adjudicator_output = adjudicator_response.data
else:
    print("No candidates to adjudicate!")

⚖️  ADJUDICATOR AGENT TEST
Original Query: I need a immigration and relocation firm that specializes in migration, business and investment migration.
Candidates to evaluate: 7
Relevance threshold: 0.7

Evaluating candidate profiles...

✅ Adjudication Completed: True
📝 Message: Successfully adjudicated 7 profiles. 6 found relevant.
⏱️  Processing Time: 49.69 seconds
🎯 Overall Confidence: 0.92

📊 Adjudication Metadata:
  - total_evaluated: 7
  - total_relevant: 6
  - total_rejected: 1
  - relevance_threshold: 0.7


In [ ]:
# Display adjudication results details
if adjudicator_output:
    print(f"\nADJUDICATION RESULTS SUMMARY")
    print("=" * 60)
    print(f"Total evaluated: {len(adjudicator_output.adjudicated_profiles)}")
    print(f"Relevant profiles: {len(adjudicator_output.relevant_profiles)}")
    print(f"Rejection summary: {adjudicator_output.rejection_summary}")
    print(f"Adjudication confidence: {adjudicator_output.adjudication_confidence:.2f}")
    
    print("\nRELEVANT PROFILES:")
    for i, profile in enumerate(adjudicator_output.relevant_profiles, 1):
        print(f"\n--- Relevant Profile {i} ---")
        print(f"ID: {profile.profile_id}")
        print(f"Original Similarity Score: {profile.similarity_score:.3f}")
        
        # Find the adjudication result for this profile
        adj_result = next((r for r in adjudicator_output.adjudicated_profiles 
                          if r.profile_match.profile_id == profile.profile_id), None)
        if adj_result:
            print(f"Relevance Score: {adj_result.relevance_score:.3f}")
            print(f"Confidence: {adj_result.confidence_level:.3f}")
            print(f"Reasoning: {adj_result.reasoning[:200]}...")
    
    print("\nREJECTED PROFILES:")
    rejected = [r for r in adjudicator_output.adjudicated_profiles if not r.is_relevant]
    for i, adj_result in enumerate(rejected[:3], 1):  # Show first 3 rejected
        print(f"\n--- Rejected Profile {i} ---")
        print(f"ID: {adj_result.profile_match.profile_id}")
        print(f"Relevance Score: {adj_result.relevance_score:.3f}")
        print(f"Reasoning: {adj_result.reasoning[:200]}...")


📋 ADJUDICATION RESULTS SUMMARY
Total evaluated: 7
Relevant profiles: 6
Rejection summary: Rejected 1 profiles. Main reasons: location mismatch (1)
Adjudication confidence: 0.92

✅ RELEVANT PROFILES:

--- Relevant Profile 1 ---
ID: 202
Original Similarity Score: 1.000
Relevance Score: 1.000
Confidence: 0.950
Reasoning: This profile is an exact match for the user's query. The service description explicitly mentions business and investment migration, which are the key requirements. The company specializes in immigrati...

--- Relevant Profile 2 ---
ID: 201
Original Similarity Score: 1.000
Relevance Score: 0.950
Confidence: 0.980
Reasoning: This profile is an excellent match for the original query about immigration and relocation services, specifically focusing on migration, business, and investment migration. Key strengths include:

1. ...

--- Relevant Profile 3 ---
ID: 172
Original Similarity Score: 0.847
Relevance Score: 0.850
Confidence: 0.900
Reasoning: The profile is highly relevan

## Step 3: Summarizer Agent Testing

The SummarizerAgent creates a comprehensive summary of the relevant profiles.

In [ ]:
# Create summarizer input using relevant profiles
if adjudicator_output and adjudicator_output.relevant_profiles:
    summarizer_input = SummarizerAgentInput(
        original_query=TEST_QUERY,
        relevant_profiles=adjudicator_output.relevant_profiles,
        summary_style="comprehensive"  # Options: 'brief', 'comprehensive', 'detailed'
    )
    
    print("SUMMARIZER AGENT TEST")
    print("=" * 60)
    print(f"Original Query: {summarizer_input.original_query}")
    print(f"Profiles to summarize: {len(summarizer_input.relevant_profiles)}")
    print(f"Summary style: {summarizer_input.summary_style}")
    print("\nGenerating summary...")
    
    # Execute summarization
    summarizer_response = await summarizer_agent.summarize(summarizer_input)
    
    # Display results
    print(f"\nSummarization Completed: {summarizer_response.success}")
    print(f"Message: {summarizer_response.message}")
    print(f"Processing Time: {summarizer_response.processing_time:.2f} seconds")
    print(f"Confidence Score: {summarizer_response.confidence_score:.2f}")
    
    print("\nSummarization Metadata:")
    for key, value in summarizer_response.metadata.items():
        if key != 'statistics':  # Show statistics separately
            print(f"  - {key}: {value}")
    
    # Store summarizer output
    summarizer_output = summarizer_response.data
else:
    print("No relevant profiles to summarize!")

📝 SUMMARIZER AGENT TEST
Original Query: I need a immigration and relocation firm that specializes in migration, business and investment migration.
Profiles to summarize: 6
Summary style: comprehensive

Generating summary...

✅ Summarization Completed: True
📝 Message: Successfully created comprehensive summary for 6 providers
⏱️  Processing Time: 12.65 seconds
🎯 Confidence Score: 0.90

📊 Summarization Metadata:
  - summary_style: comprehensive
  - profiles_summarized: 6


In [ ]:
# Display summary details
if summarizer_output:
    print("\nFINAL SUMMARY")
    print("=" * 60)
    
    print("\nEXECUTIVE SUMMARY:")
    print(summarizer_output.executive_summary)
    
    print("\nDETAILED SUMMARY:")
    print(summarizer_output.detailed_summary[:1000] + "...")  # Show first 1000 chars
    
    print("\nPROVIDER RECOMMENDATIONS:")
    for i, rec in enumerate(summarizer_output.provider_recommendations, 1):
        print(f"{i}. {rec}")
    
    print("\nKEY INSIGHTS:")
    for i, insight in enumerate(summarizer_output.key_insights, 1):
        print(f"{i}. {insight}")
    
    print("\nSUMMARY STATISTICS:")
    for key, value in summarizer_output.summary_statistics.items():
        print(f"  - {key}: {value}")


📋 FINAL SUMMARY

📌 EXECUTIVE SUMMARY:
For immigration and business/investment migration services, two standout providers emerge: AIMS Immigration & Relocation Specialist and Abode Options. Both offer comprehensive, globally-recognized migration solutions with high client satisfaction ratings and specialized expertise in business and investment migration.

📄 DETAILED SUMMARY:
The analysis reveals multiple high-quality immigration and relocation service providers with strong capabilities in business and investment migration:

1. AIMS Immigration & Relocation Specialist (Singapore):
- Comprehensive immigration services spanning skilled migration, business migration, and investment migration
- 17+ years of experience with over 300 employees
- Recognized with Top 100 Global Migration Agency CEOs award
- Offers end-to-end support including visa applications, documentation, and settlement assistance

2. Abode Options (UAE):
- Specialized in citizenship and residency by investment programs
- 

## Complete Workflow Test

Now let's test the complete workflow that orchestrates all agents automatically.

In [ ]:
# Initialize the complete workflow
workflow = GrowbalIntelligenceWorkflow(api_key=api_key)

print("COMPLETE WORKFLOW TEST")
print("=" * 60)
print(f"Query: {TEST_QUERY}")
print("\nRunning complete workflow...")

# Run the workflow with 7 max results
start_time = datetime.now()
workflow_state = await workflow.run(TEST_QUERY, max_results=7)
end_time = datetime.now()

print(f"\nWorkflow Completed!")
print(f"Total Processing Time: {workflow_state.total_processing_time:.2f} seconds")
print(f"Workflow ID: {workflow_state.workflow_id}")
print(f"Start Time: {workflow_state.start_time}")
print(f"End Time: {workflow_state.end_time}")

🔄 COMPLETE WORKFLOW TEST
Query: I need a immigration and relocation firm that specializes in migration, business and investment migration.

Running complete workflow...

✅ Workflow Completed!
⏱️  Total Processing Time: 68.56 seconds
🆔 Workflow ID: e865e39e-ad2d-4011-926a-55aea19d6f79
📅 Start Time: 2025-07-04 11:23:18.495980
📅 End Time: 2025-07-04 11:24:27.054523


In [ ]:
# Display workflow execution log
print("\nWORKFLOW EXECUTION LOG")
print("=" * 60)

for log_entry in workflow_state.agent_execution_log:
    print(f"\nAgent: {log_entry['agent']}")
    print(f"   Start: {log_entry.get('start_time', 'N/A')}")
    print(f"   End: {log_entry.get('end_time', 'N/A')}")
    print(f"   Success: {log_entry.get('success', 'N/A')}")
    print(f"   Message: {log_entry.get('message', 'N/A')}")

# Check for errors
if workflow_state.errors:
    print("\nERRORS ENCOUNTERED:")
    for error in workflow_state.errors:
        print(f"  - {error}")
else:
    print("\nNo errors encountered!")


📜 WORKFLOW EXECUTION LOG

🤖 Agent: search
   Start: 2025-07-04T11:23:18.496953
   End: 2025-07-04T11:23:26.749255
   Success: True
   Message: Successfully found 7 candidate profiles using hybrid search

🤖 Agent: adjudicator
   Start: 2025-07-04T11:23:26.749883
   End: 2025-07-04T11:24:15.111295
   Success: True
   Message: Successfully adjudicated 7 profiles. 6 found relevant.

🤖 Agent: summarizer
   Start: 2025-07-04T11:24:15.113220
   End: 2025-07-04T11:24:27.054503
   Success: True
   Message: Successfully created comprehensive summary for 6 providers

✅ No errors encountered!


In [ ]:
# Compare individual agent results with workflow results
print("\nWORKFLOW RESULTS COMPARISON")
print("=" * 60)

# Search phase comparison
if workflow_state.search_completed and workflow_state.search_output:
    print("\nSearch Phase:")
    print(f"  - Candidates found: {len(workflow_state.search_output.candidate_profiles)}")
    print(f"  - Search time: {workflow_state.search_output.search_time_seconds:.2f}s")

# Adjudication phase comparison
if workflow_state.adjudication_completed and workflow_state.adjudicator_output:
    print("\nAdjudication Phase:")
    print(f"  - Profiles evaluated: {len(workflow_state.adjudicator_output.adjudicated_profiles)}")
    print(f"  - Relevant profiles: {len(workflow_state.adjudicator_output.relevant_profiles)}")

# Summary phase comparison
if workflow_state.summary_completed and workflow_state.summarizer_output:
    print("\nSummary Phase:")
    print(f"  - Executive Summary: {'✓' if workflow_state.summarizer_output.executive_summary else '✗'}")
    print(f"  - Recommendations: {len(workflow_state.summarizer_output.provider_recommendations)}")
    print(f"  - Key Insights: {len(workflow_state.summarizer_output.key_insights)}")


🔍 WORKFLOW RESULTS COMPARISON

📍 Search Phase:
  - Candidates found: 7
  - Search time: 0.84s

📍 Adjudication Phase:
  - Profiles evaluated: 7
  - Relevant profiles: 6

📍 Summary Phase:
  - Executive Summary: ✓
  - Recommendations: 4
  - Key Insights: 4


## Testing Different Query Types

Let's test how the system handles different types of queries.

In [ ]:
# Test different query types
test_queries = [
    {
        "query": "accounting services",
        "description": "Simple tag-based query"
    },
    {
        "query": "I need a company that can help me with international trade compliance and export documentation",
        "description": "Complex natural language query"
    },
    {
        "query": "software development AND mobile apps AND iOS",
        "description": "Query with multiple specific requirements"
    },
    {
        "query": "urgent help with cybersecurity incident response",
        "description": "Urgent/time-sensitive query"
    }
]

print("TESTING DIFFERENT QUERY TYPES")
print("=" * 60)

for test_case in test_queries:
    print(f"\nTest: {test_case['description']}")
    print(f"   Query: {test_case['query']}")
    
    # Just test the search agent to see how it interprets different queries
    search_input = SearchAgentInput(query=test_case['query'], max_results=3)
    search_response = await search_agent.search(search_input)
    
    if search_response.success:
        print(f"   Strategy: {search_response.metadata.get('search_strategy')}")
        print(f"   Results: {len(search_response.data.candidate_profiles) if search_response.data else 0}")
        if search_response.metadata.get('extracted_tags'):
            print(f"   Tags: {search_response.metadata.get('extracted_tags')}")
    else:
        print(f"   Error: {search_response.message}")

🧪 TESTING DIFFERENT QUERY TYPES

📌 Test: Simple tag-based query
   Query: accounting services
   ✓ Strategy: tags
   ✓ Results: 3
   ✓ Tags: ['accounting', 'financial services']

📌 Test: Complex natural language query
   Query: I need a company that can help me with international trade compliance and export documentation
   ✓ Strategy: hybrid
   ✓ Results: 3
   ✓ Tags: ['international trade compliance', 'export documentation', 'trade consulting', 'logistics']

📌 Test: Query with multiple specific requirements
   Query: software development AND mobile apps AND iOS
   ✓ Strategy: hybrid
   ✓ Results: 3
   ✓ Tags: ['software development', 'mobile apps', 'iOS']

📌 Test: Urgent/time-sensitive query
   Query: urgent help with cybersecurity incident response
   ✓ Strategy: hybrid
   ✓ Results: 3
   ✓ Tags: ['cybersecurity', 'incident response', 'security']


## Performance Metrics Summary

Let's calculate and display overall performance metrics.

In [ ]:
# Calculate performance metrics
print("PERFORMANCE METRICS SUMMARY")
print("=" * 60)

if 'search_response' in locals() and 'adjudicator_response' in locals() and 'summarizer_response' in locals():
    total_individual_time = (
        search_response.processing_time + 
        adjudicator_response.processing_time + 
        summarizer_response.processing_time
    )
    
    print("\nIndividual Agent Times:")
    print(f"  - Search Agent: {search_response.processing_time:.2f}s")
    print(f"  - Adjudicator Agent: {adjudicator_response.processing_time:.2f}s")
    print(f"  - Summarizer Agent: {summarizer_response.processing_time:.2f}s")
    print(f"  - Total: {total_individual_time:.2f}s")
    
    if 'workflow_state' in locals():
        print("\nWorkflow Orchestration:")
        print(f"  - Total Workflow Time: {workflow_state.total_processing_time:.2f}s")
        overhead = workflow_state.total_processing_time - total_individual_time
        print(f"  - Orchestration Overhead: {overhead:.2f}s")
        print(f"  - Overhead Percentage: {(overhead/workflow_state.total_processing_time)*100:.1f}%")

print("\nThroughput Metrics:")
if 'search_output' in locals() and search_output:
    print(f"  - Profiles Searched: {search_output.total_profiles_searched}")
    print(f"  - Candidates Found: {len(search_output.candidate_profiles)}")
    if 'adjudicator_output' in locals() and adjudicator_output:
        print(f"  - Profiles Evaluated: {len(adjudicator_output.adjudicated_profiles)}")
        print(f"  - Relevant Profiles: {len(adjudicator_output.relevant_profiles)}")
        relevance_rate = len(adjudicator_output.relevant_profiles) / len(adjudicator_output.adjudicated_profiles) * 100
        print(f"  - Relevance Rate: {relevance_rate:.1f}%")

📊 PERFORMANCE METRICS SUMMARY

⏱️  Individual Agent Times:
  - Search Agent: 7.38s
  - Adjudicator Agent: 49.69s
  - Summarizer Agent: 12.65s
  - Total: 69.72s

⏱️  Workflow Orchestration:
  - Total Workflow Time: 68.56s
  - Orchestration Overhead: -1.16s
  - Overhead Percentage: -1.7%

📈 Throughput Metrics:
  - Profiles Searched: 34
  - Candidates Found: 7
  - Profiles Evaluated: 7
  - Relevant Profiles: 6
  - Relevance Rate: 85.7%


## Conclusions and Observations

This notebook has demonstrated:

1. **Individual Agent Testing**: Each agent can be tested independently with custom inputs
2. **Data Flow**: Output from one agent seamlessly becomes input for the next
3. **Workflow Orchestration**: The complete workflow manages all agents automatically
4. **Error Handling**: The system gracefully handles errors and edge cases
5. **Performance Tracking**: Detailed metrics are available at each step

Key observations:
- The SearchAgent intelligently chooses between semantic, tag-based, and hybrid search
- The AdjudicatorAgent provides detailed reasoning for relevance decisions
- The SummarizerAgent creates comprehensive, structured summaries
- The workflow adds minimal overhead while providing robust orchestration